In [1]:
from google.colab import drive
drive.mount('/content/gdrive') 
%cd /content/gdrive/MyDrive/MVA/Audio_Signal_Analysis/Project/ACVAE-VC

Mounted at /content/gdrive
/content/gdrive/MyDrive/MVA/Audio_Signal_Analysis/Project/ACVAE-VC


In [2]:
!pip install pyworld
!pip install librosa
!pip install pysptk

     |████████████████████████████████| 212 kB 8.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyworld: filename=pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl size=609490 sha256=295b365812c754d01db67bf54770866d1c7049eb8126d2cbe7ff3aa48614edbc
  Stored in directory: /root/.cache/pip/wheels/e7/7c/11/c775fffa0e1e7b05a6604b4323408a77f80fb4ab304d96b5c6
Successfully built pyworld
     |████████████████████████████████| 410 kB 7.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pysptk: filename=pysptk-0.1.20-cp37-cp37m-linux_x86_64.whl size=929938 sha256=9cbebdf5091fcffc4104efa568d76a766da7a82323caeaf798289fd2d537409d
  Stored in directory: /root/.cache/pip/wheels/dd/37/3f/7583b00bf73e73650aad5d268ee46b95dbd6dffbd237e1bf58
Successfully built pysptk


## Import

In [3]:
import soundfile as sf 

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

import os
import pyworld
import librosa
import time
import matplotlib.pyplot as plt

from preprocess import *
from model import *

torch.manual_seed(0)
np.random.seed(0)

## Data Settings

In [5]:
model_name = "model"
model_dir = "./model/" + model_name

data_dir = "./dataset/vcc2018_training"
voice_dir_list = ["VCC2SF1", "VCC2SF2", "VCC2SM2", "VCC2SM1"]

output_dir = "./converted_voices/test/" + model_name + "_training_progress"
figure_dir = "./figure/" + model_name

preprocessed_dir = "./preprocessed_dir"

## Model Settings

In [6]:
lambda_p = 70 
lambda_s = 70  
nb_label = len(voice_dir_list)

## Training Settings 

In [7]:
num_epochs = 12000
batch_size = 8
learning_rate = 1e-3
learning_rate_ = 1e-4
learning_rate__ = 1e-5
learning_rate___ = 1e-6
sampling_rate = 16000
num_envelope  = 36
num_mcep = 36
frame_period = 5.0
n_frames = 1024 

## Preprocessing

In [ ]:
for v in voice_dir_list:
    if "log_f0_"+v+".npz" in  os.listdir(os.path.join(data_dir, v)):
        continue
    print("Preprocess: " + v)
    preprocess_voice(os.path.join(data_dir, v), v,sampling_rate = 16000, num_mcep = 36, frame_period = 5.0, n_frames = 1024)

Preprocess: VCC2SF1
Preprocessing Data...
Data Loading...
Extracting f0 and mcep...
Saving f0 Data...
Saving mcep Data...
Preprocessing Done.
Time Elapsed for Data Preprocessing: 00:02:18
Preprocess: VCC2SF2
Preprocessing Data...
Data Loading...
Extracting f0 and mcep...
Saving f0 Data...
Saving mcep Data...
Preprocessing Done.
Time Elapsed for Data Preprocessing: 00:02:12
Preprocess: VCC2SM2
Preprocessing Data...
Data Loading...
Extracting f0 and mcep...
Saving f0 Data...
Saving mcep Data...
Preprocessing Done.
Time Elapsed for Data Preprocessing: 00:02:19
Preprocess: VCC2SM1
Preprocessing Data...
Data Loading...
Extracting f0 and mcep...
Saving f0 Data...
Saving mcep Data...
Preprocessing Done.
Time Elapsed for Data Preprocessing: 00:02:57


## Functions

In [8]:
def model_save(model, model_dir, model_name):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    torch.save(model.state_dict(), os.path.join(model_dir, model_name))
    
def model_load(model_dir, model_name):
    model = ACVAE(nb_label=nb_label,lambda_p=lambda_p,lambda_s=lambda_s)
    model.load_state_dict(torch.load(os.path.join(model_dir, model_name)))

In [9]:
def save_figure(losses, epoch):        
    if not os.path.exists(figure_dir):
            os.makedirs(figure_dir)
    losses = np.array(losses)
    losses = losses.reshape(-1, 4)
    x = np.linspace(0, len(losses), len(losses))
    losses_label = ("L1", "KLD", "AC_p", "AC_s")
    plt.figure()
    plt.plot(x, losses[:,0], label=losses_label[0])
    plt.plot(x, losses[:,1], label=losses_label[1])
    plt.plot(x, losses[:,2], label=losses_label[2])
    plt.plot(x, losses[:,3], label=losses_label[3])
    plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0)
    plt.savefig(figure_dir + "/" + "epoch_{:05}".format(epoch) + ".png")
    plt.savefig(figure_dir + "/" + "result.png")
        
    plt.figure()
    plt.plot(x, losses[:,2], label=losses_label[2])
    plt.plot(x, losses[:,3], label=losses_label[3])
    plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0)
    plt.savefig(figure_dir + "/" + "epoch_{:05}_AC".format(epoch) + ".png")
    plt.savefig(figure_dir + "/" + "result_AC.png")
    
    plt.figure()
    plt.plot(x, losses[:,0], label=losses_label[0])
    plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0)
    plt.savefig(figure_dir + "/" + "epoch_{:05}_L1".format(epoch) + ".png")
    plt.savefig(figure_dir + "/" + "result_L1.png")
    
    plt.figure()
    plt.plot(x, losses[:,1], label=losses_label[1])
    plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0)
    plt.savefig(figure_dir + "/" + "epoch_{:05}_KLD".format(epoch) + ".png")
    plt.savefig(figure_dir + "/" + "result_KLD.png")

In [11]:
def save_preprocessing(): 
  
  ## This function wasn't in the original repository. 
  for voice_label in voice_dir_list:
      voice_path = os.path.join(data_dir, voice_label)
      files = os.listdir(voice_path)
      vects = []
      for wav_file in files : 
        if wav_file.endswith('.wav') : 
            wav, _ = librosa.load(os.path.join(voice_path, wav_file), sr = sampling_rate, mono = True)
            wav = librosa.util.normalize(wav, norm=np.inf, axis=None)
            wav = wav_padding(wav = wav, sr = sampling_rate, frame_period = frame_period, multiple = 4)
            f0, timeaxis, sp, ap, mc = world_decompose(wav = wav, fs = sampling_rate, frame_period = frame_period, num_mcep = num_mcep)
        mc_transposed = np.array(mc).T
        frames = np.shape(mc_transposed)[1]
        cur_dir = os.path.join(preprocessed_dir, voice_label)
        dir = os.path.join(cur_dir, wav_file[:-4])
        if not os.path.exists( dir ):
          os.makedirs(dir)
        np.save(dir, mc_transposed, allow_pickle=True)
        os.rmdir(dir)

do_preprocessing = False
if do_preprocessing == True : 
  save_preprocessing()

In [12]:
def data_load(batchsize = 1, s = -1, t = -1):
    x = []
    label = []
    for i in range(batchsize):
        if (s == -1):
            label_num = np.random.randint(nb_label)
        else:
            label_num = s
        voice_path = os.path.join(data_dir, voice_dir_list[label_num])
        voice_path_npy = os.path.join(preprocessed_dir, voice_dir_list[label_num])
        files = os.listdir(voice_path_npy)
        frames = 0
        while frames < n_frames:
            
            file = ""
            while file.count("npy") == 0:
                file = np.random.choice(files)
            mc_transposed = np.load(os.path.join(voice_path_npy, file))
            frames = mc_transposed.shape[1]
        mcep_normalization_params = np.load(os.path.join(voice_path, "mcep_"+voice_dir_list[label_num]+".npz"))
        mcep_mean = mcep_normalization_params['mean']
        mcep_std = mcep_normalization_params['std']
        mc_norm = (mc_transposed  - mcep_mean) / mcep_std
  
        start_ = np.random.randint(frames - n_frames + 1)
        end_ = start_ + n_frames

        x.append(mc_norm[:,start_:end_])
        label.append(label_num)
    return torch.Tensor(x).view(batchsize, 1, num_mcep, n_frames), torch.Tensor(label)

In [13]:
def test_conv(model, epoch):
    print("Test")
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    output_epoch_dir = os.path.join(output_dir, "epoch_{:05}".format(epoch))
    if not os.path.exists(output_epoch_dir):
        os.makedirs(output_epoch_dir)
    
    for s_label in range(nb_label):
    
        voice_path_s = os.path.join(data_dir, voice_dir_list[s_label])

        files = os.listdir(voice_path_s)
        file = ""
        while file.count("wav") == 0:
            file = np.random.choice(files)
        
        print("Source File:" + file)

        for t_label in range(nb_label):

            if (t_label == s_label):
                continue

            voice_path_t = os.path.join(data_dir, voice_dir_list[t_label])

            wav, _ = librosa.load(os.path.join(voice_path_s, file), sr = sampling_rate, mono = True)
            wav = librosa.util.normalize(wav, norm=np.inf, axis=None)
            wav = wav_padding(wav = wav, sr = sampling_rate, frame_period = frame_period, multiple = 4)
            f0, timeaxis, sp, ap, mc = world_decompose(wav = wav, fs = sampling_rate, frame_period = frame_period)

            mc_transposed  = np.array(mc).T

            mcep_normalization_params_s = np.load(os.path.join(voice_path_s, "mcep_"+voice_dir_list[s_label]+".npz"))
            mcep_mean_s = mcep_normalization_params_s['mean']
            mcep_std_s = mcep_normalization_params_s['std']    
            mcep_normalization_params_t = np.load(os.path.join(voice_path_t, "mcep_"+voice_dir_list[t_label]+".npz"))
            mcep_mean_t = mcep_normalization_params_t['mean']
            mcep_std_t = mcep_normalization_params_t['std']

            mc_norm = (mc_transposed - mcep_mean_s) / mcep_std_s

            x = torch.Tensor(mc_norm).view(1, 1, mc_norm.shape[0], mc_norm.shape[1])
            x = x.to(device)

            label_s_tensor = torch.Tensor(np.array([s_label])).view(1, 1).to(device)
            label_t_tensor = torch.Tensor(np.array([t_label])).view(1, 1).to(device)

            mu_enc, logvar_enc = model.encode(x, label_s_tensor)
            z_enc = model.reparameterize(mu_enc, logvar_enc)

            mu_dec_t, logvar_dec_t = model.decode(z_enc, label_t_tensor)
            z_dec_t = model.reparameterize(mu_dec_t, logvar_dec_t)
            if (torch.cuda.is_available()):
                z_dec_t = z_dec_t.data.cpu().numpy().reshape((z_dec_t.shape[2], z_dec_t.shape[3]))
            else:
                z_dec_t = z_dec_t.data.numpy().reshape((z_dec_t.shape[2], z_dec_t.shape[3]))

            mu_dec_s, logvar_dec_s = model.decode(z_enc, label_s_tensor)
            z_dec_s = model.reparameterize(mu_dec_s, logvar_dec_s)
            if (torch.cuda.is_available()):
                z_dec_s = z_dec_s.data.cpu().numpy().reshape((z_dec_s.shape[2], z_dec_s.shape[3]))
            else:
                z_dec_s = z_dec_s.data.numpy().reshape((z_dec_s.shape[2], z_dec_s.shape[3]))

            mc_converted_t = z_dec_t * mcep_std_t + mcep_mean_t
            mc_converted_t = mc_converted_t.T
            mc_converted_t = np.ascontiguousarray(mc_converted_t)
            sp_converted_t = world_decode_mc(mc = mc_converted_t, fs = sampling_rate)
            mc_converted_s = z_dec_s * mcep_std_s + mcep_mean_s
            mc_converted_s = mc_converted_s.T
            mc_converted_s = np.ascontiguousarray(mc_converted_s)
            sp_converted_s = world_decode_mc(mc = mc_converted_s, fs = sampling_rate)

            sp_gained = np.multiply(sp, np.divide(sp_converted_t, sp_converted_s))

            logf0s_normalization_params_s = np.load(os.path.join(voice_path_s, "log_f0_"+voice_dir_list[s_label]+".npz"))
            logf0s_mean_s = logf0s_normalization_params_s['mean']
            logf0s_std_s = logf0s_normalization_params_s['std']
            logf0s_normalization_params_t = np.load(os.path.join(voice_path_t, "log_f0_"+voice_dir_list[t_label]+".npz"))
            logf0s_mean_t = logf0s_normalization_params_t['mean']
            logf0s_std_t = logf0s_normalization_params_t['std']

            f0_converted = pitch_conversion(f0 = f0, mean_log_src = logf0s_mean_s, std_log_src = logf0s_std_s, mean_log_target = logf0s_mean_t, std_log_target = logf0s_std_t)
            
            wav_transformed = world_speech_synthesis(f0 = f0_converted, sp = sp_gained, ap = ap, fs = sampling_rate, frame_period = frame_period)
            sf.write(os.path.join(output_epoch_dir, voice_dir_list[s_label]+"_to_"+voice_dir_list[t_label]+"_["+file+"].wav"), wav_transformed, sampling_rate)
            wav_source = world_speech_synthesis(f0 = f0_converted, sp = sp, ap = ap, fs = sampling_rate, frame_period = frame_period)
            sf.write(os.path.join(output_epoch_dir, voice_dir_list[s_label]+"_to_"+voice_dir_list[t_label]+"_["+file+"]_nonconv.wav"), wav_source, sampling_rate)

            print("Converted: " + voice_dir_list[s_label] + " -> " + voice_dir_list[t_label])

## Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = ACVAE(nb_label=nb_label,lambda_p=lambda_p,lambda_s=lambda_s).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.train()

losses = []

for epoch in range(num_epochs):
    epoch += 1
    
    if (epoch == 3000):
        learning_rate = learning_rate_ 
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    if (epoch == 6000):
        learning_rate = learning_rate__
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    if (epoch == 8000):
        learning_rate = learning_rate___
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    
    start_time = time.time()

    print('Epoch: %d' % epoch)

    x_, label_ = data_load(batch_size)
    optimizer.zero_grad()
    loss, loss_list = model.calc_loss(x_, label_)
    print('loss : ', loss)
    loss.backward()
    losses.append(loss_list)
    optimizer.step()
    
    if epoch % 500 == 0:
        test_conv(model, epoch)
    if epoch % 100 == 0:
        model_save(model, model_dir, model_name)
    if epoch % 2000 == 0:
        model_save(model, model_dir, model_name + "_" + str(epoch))

    if epoch % 100 == 0:
        save_figure(losses, epoch)
    
    elapsed_time = time.time() - start_time
    print('Time Elapsed for one epoch: %02d:%02d:%02d' % (elapsed_time // 3600, (elapsed_time % 3600 // 60), (elapsed_time % 60 // 1)))

model_save(model, model_dir, model_name)

save_figure(losses, epoch)


cuda
Epoch: 1
loss :  tensor(487328.7188, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 2
loss :  tensor(439199.4062, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 3
loss :  tensor(419388.1250, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 4
loss :  tensor(398405.2812, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 5
loss :  tensor(380692.4062, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 6
loss :  tensor(370296.2500, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 7
loss :  tensor(358763., device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 8
loss :  tensor(349267.3750, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 9
loss :  tensor(355256.4688, device='cuda:0', grad_fn=<AddBackward0>)
T

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:200: RuntimeWarning: divide by zero encountered in log


Converted: VCC2SF1 -> VCC2SF2
Converted: VCC2SF1 -> VCC2SM2
Converted: VCC2SF1 -> VCC2SM1
Source File:10016.wav
Converted: VCC2SF2 -> VCC2SF1
Converted: VCC2SF2 -> VCC2SM2
Converted: VCC2SF2 -> VCC2SM1
Source File:10070.wav
Converted: VCC2SM2 -> VCC2SF1
Converted: VCC2SM2 -> VCC2SF2
Converted: VCC2SM2 -> VCC2SM1
Source File:10010.wav
Converted: VCC2SM1 -> VCC2SF1
Converted: VCC2SM1 -> VCC2SF2
Converted: VCC2SM1 -> VCC2SM2
Time Elapsed for one epoch: 00:00:38
Epoch: 501
loss :  tensor(219480.6094, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 502
loss :  tensor(219185.0625, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 503
loss :  tensor(219446.8750, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 504
loss :  tensor(221290.4219, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 505
loss :  tensor(224009., device='cuda:0', grad_fn=<AddBackw

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning

Time Elapsed for one epoch: 00:00:01
Epoch: 601
loss :  tensor(219026.2188, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 602
loss :  tensor(217088.0625, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 603
loss :  tensor(211827.1875, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 604
loss :  tensor(220752.9062, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 605
loss :  tensor(215057.6562, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 606
loss :  tensor(219002.1406, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 607
loss :  tensor(215942.9688, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 608
loss :  tensor(216870.1562, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 609
loss :  tensor(2

...

Time Elapsed for one epoch: 00:00:01
Epoch: 11301
loss :  tensor(205060., device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11302
loss :  tensor(197371.1719, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11303
loss :  tensor(206602.1250, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11304
loss :  tensor(195763.2969, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11305
loss :  tensor(198593.8281, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11306
loss :  tensor(208608.8281, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11307
loss :  tensor(201428.2500, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11308
loss :  tensor(204015.7188, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11309
lo

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning

Time Elapsed for one epoch: 00:00:01
Epoch: 11401
loss :  tensor(202326.4844, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11402
loss :  tensor(196975.1875, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11403
loss :  tensor(204901.2969, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11404
loss :  tensor(200958.8750, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11405
loss :  tensor(197175.8125, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11406
loss :  tensor(199507.3906, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11407
loss :  tensor(196742.3438, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 11408
loss :  tensor(202441.8906, device='cuda:0', grad_fn=<AddBackward0>)
Time Elapsed for one epoch: 00:00:00
Epoch: 1140

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:200: RuntimeWarning: divide by zero encountered in log


Converted: VCC2SF1 -> VCC2SF2
Converted: VCC2SF1 -> VCC2SM2
Converted: VCC2SF1 -> VCC2SM1
Source File:10001.wav
Converted: VCC2SF2 -> VCC2SF1
Converted: VCC2SF2 -> VCC2SM2
Converted: VCC2SF2 -> VCC2SM1
Source File:10003.wav
Converted: VCC2SM2 -> VCC2SF1
